In [1]:
import tensorflow as tf
from makiflow.models.classificator import Classificator
from makiflow.metrics import bin_categorical_dice_coeff
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import pandas as pd
import json
from tqdm import tqdm
%matplotlib inline

from makiflow import set_main_gpu
set_main_gpu(';')

In [2]:
N_CLASSES = 8
IMAGE_SHAPE = (1024, 1024)

In [3]:
SESSION = tf.Session()

In [33]:
def create_model(arch_path, weights_path):
    model = Classificator.from_json(arch_path)
    model.set_session(SESSION)
    model.load_weights(weights_path)
    return model

def load_im_mask(image_path, masks_folder_path):
    image_shape = IMAGE_SHAPE
    n_classes = N_CLASSES
    image = cv2.imread(image_path)

    mask_folder = masks_folder_path

    label_tensor = np.zeros(shape=(*image_shape, n_classes + 1), dtype='int32')
    for binary_mask_path in glob(os.path.join(mask_folder, '*')):
        filename = binary_mask_path.split('/')[-1]
        class_id = int(filename.split('.')[0])
        assert class_id != 0, 'Encountered class 0. Class names must start from 1.'
        binary_mask = cv2.imread(binary_mask_path)
        assert binary_mask is not None, f'Could not load mask with name={binary_mask_path}'
        label_tensor[..., class_id] = binary_mask[..., 0] * class_id
    label_tensor = np.max(label_tensor, axis=-1)
    
    return image.astype(np.float32, copy=False) / 255, label_tensor

def load_data(config_path):
    config = load_json(config_path)
    config = config['test_config']
    image_paths = config['test_image']
    mask_paths = config['test_mask']
    
    images = []
    masks = []
    for image_path, mask_folder_path in zip(image_paths, mask_paths):
        image, mask = load_im_mask(image_path, mask_folder_path)
        images.append(image)
        masks.append(mask)
    return images, masks

def load_json(path):
    with open(path, 'r') as f:
        return json.loads(f.read())

def test(arch_path, weights_path, config_path, th=0.5):
    images, masks = load_data(config_path)
    model = create_model(arch_path, weights_path)
    
    predictions = []
    max_preds = []
    for image in tqdm(images):
        pred = (model.predict([image])[0] > th).astype('uint8') * np.arange(1, 9, 1).reshape(1, 1, 8)
#         pred = pred.max(axis=-1)
        predictions += [pred]
        max_preds += [pred.max(axis=-1)]
    
    # predictions = np.concatenate(predictions, axis=0)
    f1_scores = f1_score(np.asarray(masks).reshape(-1), np.asarray(max_preds).reshape(-1), average=None)
    return confusion_matrix(np.asarray(masks).reshape(-1), np.asarray(max_preds).reshape(-1)), f1_scores


def compute_tpr_tnr_class(class_id, C):
    TN = C[0, 0]
    TP = C[class_id, class_id]
    
    FP = 0
    for i in range(0, C.shape[0]):
        if i == class_id:
            continue
        FP += C[i, class_id]
        
    FN = 0
    for i in range(0, C.shape[0]):
        if i == class_id:
            continue
        FN += C[class_id, i]
    
    tpr = 1 - FN / (FN + TP + 1e-7)
    tnr = 1 - FP / (FP + TN + 1e-7)
    acc = (TP + TN) / (TP + TN + FP + FN)
    return tpr, tnr, acc
    
def compute_tpr(C):
    trps = []
    tnrs = []
    accs = []
    for i in range(1, C.shape[0]):
        rates = compute_tpr_tnr_class(i, C)
        trps.append(rates[0])
        tnrs.append(rates[1])
        accs.append(rates[2])
    return trps, tnrs, accs

In [5]:
cv3_f0_model_arch = 'cv3_fold0/result_1/model.json'
cv3_f0_model_weights = 'cv3_fold0/result_1/epoch_13/weights.ckpt'
cv3_f0_config = 'cv3_fold0/config.json'

cv3_f0_C, f1_scores_1 = test(cv3_f0_model_arch, cv3_f0_model_weights, cv3_f0_config, 0.5)



Model is restored!


INFO:tensorflow:Restoring parameters from cv3_fold0/result_1/epoch_13/weights.ckpt


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Weights are loaded.



1it [00:05,  5.64s/it]
 12%|█▎        | 1/8 [00:07<00:51,  7.41s/it]
0it [00:00, ?it/s]
1it [00:03,  3.43s/it]
 25%|██▌       | 2/8 [00:13<00:42,  7.15s/it]
0it [00:00, ?it/s]
1it [00:03,  3.73s/it]
 38%|███▊      | 3/8 [00:18<00:31,  6.32s/it]
0it [00:00, ?it/s]
1it [00:03,  3.96s/it]
 50%|█████     | 4/8 [00:25<00:26,  6.62s/it]
0it [00:00, ?it/s]
1it [00:04,  4.07s/it]
 62%|██████▎   | 5/8 [00:32<00:20,  6.68s/it]
0it [00:00, ?it/s]
1it [00:03,  3.20s/it]
 75%|███████▌  | 6/8 [00:39<00:13,  6.63s/it]
0it [00:00, ?it/s]
1it [00:03,  3.08s/it]
 88%|████████▊ | 7/8 [00:45<00:06,  6.55s/it]
0it [00:00, ?it/s]
1it [00:02,  2.92s/it]
100%|██████████| 8/8 [00:51<00:00,  6.43s/it]


In [34]:
tpr0, tnr0, accs0 = np.array(compute_tpr(cv3_f0_C)).round(2)

In [7]:
tpr0

array([0.98, 0.86, 0.91, 0.81, 0.18, 0.25, 0.14, 0.87])

In [8]:
cv3_f1_model_arch = 'cv3_fold1/result/model.json'
cv3_f1_model_weights = 'cv3_fold1/result/epoch_13/weights.ckpt'
cv3_f1_config = 'cv3_fold1/config.json'

cv3_f1_dice, f1_scores_2 = test(cv3_f1_model_arch, cv3_f1_model_weights, cv3_f1_config, 0.5)

Model is restored!
INFO:tensorflow:Restoring parameters from cv3_fold1/result/epoch_13/weights.ckpt


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Weights are loaded.



1it [00:05,  5.64s/it]
 12%|█▎        | 1/8 [00:05<00:39,  5.71s/it]
0it [00:00, ?it/s]
1it [00:03,  3.29s/it]
 25%|██▌       | 2/8 [00:13<00:38,  6.38s/it]
0it [00:00, ?it/s]
1it [00:03,  3.22s/it]
 38%|███▊      | 3/8 [00:19<00:31,  6.35s/it]
0it [00:00, ?it/s]
1it [00:03,  3.11s/it]
 50%|█████     | 4/8 [00:23<00:21,  5.44s/it]
0it [00:00, ?it/s]
1it [00:03,  3.21s/it]
 62%|██████▎   | 5/8 [00:26<00:14,  4.85s/it]
0it [00:00, ?it/s]
1it [00:03,  3.33s/it]
 75%|███████▌  | 6/8 [00:30<00:08,  4.47s/it]
0it [00:00, ?it/s]
1it [00:03,  3.07s/it]
 88%|████████▊ | 7/8 [00:33<00:04,  4.11s/it]
0it [00:00, ?it/s]
1it [00:03,  3.18s/it]
100%|██████████| 8/8 [00:41<00:00,  5.16s/it]


In [35]:
tpr1, tnr1, accs1 = np.array(compute_tpr(cv3_f1_dice)).round(2)

In [10]:
tpr1

array([0.98, 0.72, 0.92, 0.81, 0.79, 0.03, 0.48, 0.95])

In [11]:
cv3_f2_model_arch = 'cv3_fold2/result_1/model.json'
cv3_f2_model_weights = 'cv3_fold2/result_1/epoch_13/weights.ckpt'
cv3_f2_config = 'cv3_fold2/config.json'

cv3_f2_dice, f1_scores_3 = test(cv3_f2_model_arch, cv3_f2_model_weights, cv3_f2_config, 0.5)

Model is restored!
INFO:tensorflow:Restoring parameters from cv3_fold2/result_1/epoch_13/weights.ckpt


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Weights are loaded.



1it [00:06,  6.82s/it]
 12%|█▎        | 1/8 [00:07<00:49,  7.12s/it]
0it [00:00, ?it/s]
1it [00:03,  3.27s/it]
 25%|██▌       | 2/8 [00:13<00:40,  6.79s/it]
0it [00:00, ?it/s]
1it [00:02,  2.94s/it]
 38%|███▊      | 3/8 [00:18<00:31,  6.39s/it]
0it [00:00, ?it/s]
1it [00:03,  3.28s/it]
 50%|█████     | 4/8 [00:23<00:23,  5.94s/it]
0it [00:00, ?it/s]
1it [00:02,  2.90s/it]
 62%|██████▎   | 5/8 [00:28<00:17,  5.80s/it]
0it [00:00, ?it/s]
1it [00:02,  2.96s/it]
 75%|███████▌  | 6/8 [00:34<00:11,  5.68s/it]
0it [00:00, ?it/s]
1it [00:03,  3.09s/it]
 88%|████████▊ | 7/8 [00:40<00:05,  5.67s/it]
0it [00:00, ?it/s]
1it [00:03,  3.04s/it]
100%|██████████| 8/8 [00:45<00:00,  5.73s/it]


In [36]:
tpr2, tnr2, accs2 = np.array(compute_tpr(cv3_f2_dice)).round(2)

In [13]:
f1_scores_1 = f1_scores_1[1:].round(2)
f1_scores_1 = np.hstack([f1_scores_1, np.mean(f1_scores_1).round(2)])
f1_scores_2 = f1_scores_2[1:].round(2)
f1_scores_2 = np.hstack([f1_scores_2, np.mean(f1_scores_2).round(2)])
f1_scores_3 = f1_scores_3[1:].round(2)
f1_scores_3 = np.hstack([f1_scores_3, np.mean(f1_scores_3).round(2)])

In [37]:
tpr0 = np.hstack([tpr0, np.mean(tpr0).round(2)])
tpr1 = np.hstack([tpr1, np.mean(tpr1).round(2)])
tpr2 = np.hstack([tpr2, np.mean(tpr2).round(2)])

tnr0 = np.hstack([tnr0, np.mean(tnr0).round(2)])
tnr1 = np.hstack([tnr1, np.mean(tnr1).round(2)])
tnr2 = np.hstack([tnr2, np.mean(tnr2).round(2)])

accs0 = np.hstack([accs0, np.mean(accs0).round(2)])
accs1 = np.hstack([accs1, np.mean(accs1).round(2)])
accs2 = np.hstack([accs2, np.mean(accs2).round(2)])

In [39]:
mean_tpr = np.mean([tpr0, tpr1, tpr2], axis=0).round(2)
mean_tnr = np.mean([tnr0, tnr1, tnr2], axis=0).round(2)
mean_acc = np.mean([accs0, accs1, accs2], axis=0).round(2)
mean_tpr, mean_tnr, mean_acc

(array([0.97, 0.73, 0.92, 0.83, 0.51, 0.1 , 0.32, 0.89, 0.66]),
 array([0.98, 0.99, 0.74, 0.99, 1.  , 1.  , 1.  , 0.95, 0.96]),
 array([0.98, 0.99, 0.76, 0.99, 1.  , 1.  , 0.99, 0.95, 0.96]))

In [44]:
ba = (mean_tpr + mean_tnr) / 2
ba

array([0.975, 0.86 , 0.83 , 0.91 , 0.755, 0.55 , 0.66 , 0.92 , 0.81 ])

In [20]:
mean_f1 = np.mean([f1_scores_1, f1_scores_2, f1_scores_3], axis=0).round(2)
mean_f1

array([0.78, 0.67, 0.48, 0.52, 0.47, 0.14, 0.29, 0.38, 0.46])

In [45]:
df = pd.DataFrame(
    data=np.stack([tpr0, tpr1, tpr2, f1_scores_1, f1_scores_2, f1_scores_3, mean_tpr, mean_tnr, mean_f1, mean_acc, ba]), 
    index=['test0 tpr', 'test1 tpr', 'test2 tpr', 'test0 f1',
           'test1 f1', 'test2 f1', 'mean_tpr', 'mean_tnr', 'mean_f1', 'mean_acc', 'ba'],
    columns=list(range(1,9)) + ['mean']
)

In [46]:
df.to_csv('tpr_unbalanced_p0.5.csv')